In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import os, datetime, pytz, requests
import numpy as np
from cookie import cookies, headers
from concurrent.futures import ThreadPoolExecutor

In [2]:
def generate_params(year, ctyid):
    start_date = f'01/01/{year} 00:00:00'
    end_date = f'12/31/{year} 23:59:00'

    gmt = pytz.timezone('GMT')  # Use GMT timezone
    start_datetime = gmt.localize(datetime.datetime.strptime(start_date, '%m/%d/%Y %H:%M:%S'))
    end_datetime = gmt.localize(datetime.datetime.strptime(end_date, '%m/%d/%Y %H:%M:%S'))

    s_time_id = int(start_datetime.timestamp())
    e_time_id = int(end_datetime.timestamp())

    params = {
        'report_form': '1',
        'dnode': 'County',
        'content': 'loops',
        'tab': 'det_summary',
        'export': 'xls',
        'county_id': f'{ctyid}',
        's_time_id': str(s_time_id),
        's_time_id_f': start_datetime.strftime('%m/%d/%Y'),
        'e_time_id': str(e_time_id),
        'e_time_id_f': end_datetime.strftime('%m/%d/%Y'),
        'tod': 'all',
        'tod_from': '0',
        'tod_to': '0',
        'dow_0': 'on',
        'dow_1': 'on',
        'dow_2': 'on',
        'dow_3': 'on',
        'dow_4': 'on',
        'dow_5': 'on',
        'dow_6': 'on',
        'holidays': 'on',
        'gb': 'fwy',
        'q': 'vmt',
        'q2': 'vht',
    }

    return params

In [3]:
# def printfile():
#     output_direct = '/Users/joelt/datahere/filedump'

#     for ctyid in range(7, 8):
#         for year in range(2023, 2024):
#             params = generate_params(year, ctyid)
#             response = requests.get('https://pems.dot.ca.gov/', params=params, cookies=cookies, headers=headers)

#             # Save the response content to a file
#             filename = os.path.join(output_direct, f'ctyid{ctyid}_{year}.xlsx')
#             with open(filename, 'wb') as file:
#                 file.write(response.content)

#             print(f'{filename} created successfully.')

# printfile()

In [4]:
# def download_file(year, ctyid):
#     output_direct = '/Users/joelt/datahere/filedump'
#     params = generate_params(year, ctyid)
#     with requests.Session() as session:
#         response = session.get('https://pems.dot.ca.gov/', params=params, cookies=cookies, headers=headers)
#     filename = os.path.join(output_direct, f'ctyid{ctyid}_{year}.xlsx')
#     with open(filename, 'wb') as file:
#         file.write(response.content)
#     print(f'{filename} created successfully.')

# with ThreadPoolExecutor(max_workers=5) as executor:
#     for ctyid in range(7, 116, 2):
#         for year in range(2001, 2024):
#             executor.submit(download_file, year, ctyid)


In [13]:
def get_file_size(url, params, cookies, headers):
    response = requests.head(url, params=params, cookies=cookies, headers=headers)
    file_size = int(response.headers.get('content-length', 0))
    return file_size

def download_file(year, ctyid, cookies, headers):
    output_direct = '/Users/joelt/datahere/filedump'
    params = generate_params(year, ctyid)
    url = 'https://pems.dot.ca.gov/'
    file_size = get_file_size(url, params, cookies, headers)

    # If the file size is less than 6500 bytes, it's likely the file is empty
    if file_size > 6503:
        with requests.Session() as session:
            response = session.get(url, params=params, cookies=cookies, headers=headers)
        filename = os.path.join(output_direct, f'ctyid{ctyid}_{year}.xlsx')
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f'{filename} created successfully.')
    else:
        print(f'Skipped empty file: ctyid{ctyid}_{year}.xlsx')

with ThreadPoolExecutor(max_workers=10) as executor:
    for ctyid in range(29, 116, 2):
        for year in range(2001, 2024):
            executor.submit(download_file, year, ctyid, cookies, headers)

KeyboardInterrupt: 

In [10]:
def process_files():
    input_direct = '/Users/joelt/datahere/filedump'
    output_direct = '/Users/joelt/datahere/countyVMT_files'
    
    for ctyid in range(1, 156, 2):
        yearly_data = []
        for year in range(2001, 2024):
            filename = os.path.join(input_direct, f'ctyid{ctyid}_{year}.xlsx')
            if os.path.exists(filename):
                df = pd.read_excel(filename)
                total_vmt = df['VMT'].sum()
                total_vht = df['VHT'].sum()
            else:
                total_vmt = np.nan
                total_vht = np.nan
            yearly_data.append({'Year': year, 'Total VMT': total_vmt, 'Total VHT': total_vht})
        
        result_df = pd.DataFrame(yearly_data)
        result_filename = os.path.join(output_direct, f'ctyid{ctyid}_summary.xlsx')
        result_df.to_excel(result_filename)
        print(f'{result_filename} created successfully.')

process_files()


/Users/joelt/datahere/countyVMT_files/ctyid1_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid3_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid5_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid7_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid9_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid11_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid13_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid15_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid17_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid19_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid21_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_files/ctyid23_summary.xlsx created successfully.
/Users/joelt/datahere/countyVMT_f